In [1]:
from google.colab.drive import mount
mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
drive_path = 'drive/My Drive/'

In [9]:
!cp 'drive/My Drive/IRMA.zip' IRMA.zip
!unzip 'IRMA.zip'

Archive:  IRMA.zip
   creating: IRMA/
   creating: IRMA/test/
   creating: IRMA/test/Type_D/
 extracting: IRMA/test/Type_D/1000049_cut_797990.png  
 extracting: IRMA/test/Type_D/1000511_cut_798067.png  
 extracting: IRMA/test/Type_D/1001307_cut_798986.png  
  inflating: IRMA/test/Type_D/1001562_cut_799973.png  
 extracting: IRMA/test/Type_D/1001873_cut_797921.png  
 extracting: IRMA/test/Type_D/1002697_cut_806166.png  
 extracting: IRMA/test/Type_D/1002702_cut_803768.png  
 extracting: IRMA/test/Type_D/1002758_cut_805788.png  
 extracting: IRMA/test/Type_D/1002767_cut_804211.png  
 extracting: IRMA/test/Type_D/1002921_cut_805389.png  
 extracting: IRMA/test/Type_D/1002934_cut_803708.png  
 extracting: IRMA/test/Type_D/1002960_cut_803634.png  
 extracting: IRMA/test/Type_D/1003094_cut_806456.png  
 extracting: IRMA/test/Type_D/1003108_cut_806083.png  
 extracting: IRMA/test/Type_D/1003138_cut_803616.png  
 extracting: IRMA/test/Type_D/1003212_cut_806565.png  
 extracting: IRMA/test/Type

In [0]:
import os
import numpy as np 
import matplotlib.pyplot as plt
import cv2
from random import shuffle
import glob
from numpy.linalg import norm 
import pandas as pd
import csv

In [0]:
images = glob.glob('INbreast/AllDICOMs/*.dcm')

In [0]:
images = glob.glob('IRMA/train/**/*.png')+glob.glob('IRMA/test/**/*.png')

In [0]:
IMG_SIZE = 128

In [0]:
import skimage.feature as ft


def map_label(path):
  if 'Type_D' in path:
        return 0
  if 'Type_E' in path:
      return 1
  if 'Type_F' in path:
      return 2
  if 'Type_G' in path:
      return 3
    
def preprocess(img):
  img = cv2.resize(img, dsize = (IMG_SIZE, IMG_SIZE))
  #img = np.stack((img,)*3, axis=-1)
  #img = np.expand_dims(img, -1)
  return img.astype('float32')

In [0]:
features = []
labels = []

added_so_far = []
for path  in images:
  img = cv2.imread(path)

  #get the mc mask 
  label =  map_label(path)

  processed_img = preprocess(img)/255. 
  
  #print(hog_feature.shape, lbp_feature.shape)
  features.append(np.concatenate([processed_img], axis = -1))
  labels.append(label)

In [0]:
import keras
from keras.callbacks import Callback
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten,\
 Conv2D, MaxPooling2D, Input, Concatenate, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np
import keras.backend as K
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score , f1_score, roc_auc_score       
from sklearn.model_selection import StratifiedKFold, KFold
from keras.optimizers import Adam

In [0]:
def conv_block(fs, x):
  conv  = Conv2D(fs, (3, 3), padding = 'same', activation = 'relu')(x)
  pool  = MaxPooling2D(pool_size = (2, 2))(conv)
  return pool

def res_block(fs, x):
  conv  = Conv2D(fs, (3, 3), padding = 'same', activation = 'relu')(x)
  pool  = MaxPooling2D(pool_size = (2, 2))(Concatenate(axis = -1)([x, conv]))
  return pool

def to_labels(pred):
  return np.argmax(pred, axis = 1)

def model1():
  inp = Input(shape = (IMG_SIZE, IMG_SIZE, 3))

  block1 = conv_block(32, inp)
  block2 = conv_block(64, block1)
  block3 = conv_block(64, block2)
  block4 = conv_block(128, block3)
  block5 = conv_block(128, block4)

  flatten = Flatten()(block5)

  out = Dense(128, activation = 'relu')(flatten)
  out = Dense(4, activation = 'softmax')(out)

  adam = Adam(lr = 0.0001)
  model = keras.models.Model(inputs = inp, outputs = out)
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
  return model

def resnet():
  inp = Input(shape = (IMG_SIZE, IMG_SIZE, 3))

  block1 = res_block(32, inp)
  block2 = res_block(64, block1)
  block3 = res_block(64, block2)
  block4 = res_block(128, block3)
  block5 = res_block(128, block4)
  flatten = Flatten()(block5)

  out = Dense(128, activation = 'relu')(flatten)
  out = Dense(4, activation = 'softmax')(out)

  model = keras.models.Model(inputs = inp, outputs = out)
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
  return model

def fine_tune_vgg():
  #download and load the pretrained model
  vgg = keras.applications.VGG16(input_shape = (IMG_SIZE, IMG_SIZE, 3), weights ='imagenet', include_top = False)

  #freeze all the layers as a fixed feature extractor
  for layer in vgg.layers:
    layer.trainable = False

  flatten = GlobalAveragePooling2D()(vgg.output)

  #first branch for classification
  branch1 = Dense(1024, activation = 'relu')(flatten)
  out = Dense(4, activation = 'softmax')(branch1)

  #create a model with one input and two outputs 
  model = keras.models.Model(inputs = vgg.input , outputs = out)
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [0]:
kfold = KFold(n_splits=10, shuffle=True)
X = np.array(features)
Y = np.array(labels)

In [41]:
cvscores = {'acc':[], 'f_1':[], 'prc':[], 'rec':[], 'roc':[0]}

for i, (train, test) in enumerate(kfold.split(X, Y)):
  #model = model1()
  model = resnet()
  #model = fine_tune_vgg()
  #model = fine_tune_incept()
  #model.fit(X[train], Y[train], batch_size = 64, epochs=80, verbose = 0) #resnet model 
  model.fit(X[train], Y[train], batch_size = 32, epochs=50, verbose = 2) #model1  
  # evaluate the model
  logits = model.predict(X[test])
  pred = to_labels(logits)
  s1 = accuracy_score(Y[test], pred)
  s2 = f1_score(Y[test], pred, average='weighted')
  s3 = precision_score(Y[test], pred, average='weighted')
  s4 = recall_score(Y[test], pred, average='weighted')
  #s5 = roc_auc_score(Y[test], logits)
  
  print(f'{i}: acc:{s1:.2f}  f_1:{s2:.2f}  prc:{s3:.2f}  rec:{s4:.2f}')

  cvscores['acc'].append(s1)
  cvscores['f_1'].append(s2)
  cvscores['prc'].append(s3)
  cvscores['rec'].append(s4)
  #cvscores['roc'].append(s5)

print('======================================')
avgs = [np.mean(cvscores['acc']), np.mean(cvscores['f_1']), np.mean(cvscores['prc']), np.mean(cvscores['rec']), np.mean(cvscores['roc'])]
print(f'acc:{avgs[0]:.2f}  f_1:{avgs[1]:.2f}  prc:{avgs[2]:.2f}  rec:{avgs[3]:.2f} roc:{avgs[4]:.2f}')

Epoch 1/50
 - 5s - loss: 1.4277 - acc: 0.2603
Epoch 2/50
 - 2s - loss: 1.3724 - acc: 0.3051
Epoch 3/50
 - 2s - loss: 1.3670 - acc: 0.3159
Epoch 4/50
 - 2s - loss: 1.3650 - acc: 0.3175
Epoch 5/50
 - 2s - loss: 1.3610 - acc: 0.3267
Epoch 6/50
 - 2s - loss: 1.3630 - acc: 0.3163
Epoch 7/50
 - 2s - loss: 1.3620 - acc: 0.3279
Epoch 8/50
 - 2s - loss: 1.3584 - acc: 0.3295
Epoch 9/50
 - 2s - loss: 1.3564 - acc: 0.3327
Epoch 10/50
 - 2s - loss: 1.3596 - acc: 0.3171
Epoch 11/50
 - 2s - loss: 1.3576 - acc: 0.3351
Epoch 12/50
 - 2s - loss: 1.3541 - acc: 0.3319
Epoch 13/50
 - 2s - loss: 1.3555 - acc: 0.3291
Epoch 14/50
 - 2s - loss: 1.3506 - acc: 0.3267
Epoch 15/50
 - 2s - loss: 1.3507 - acc: 0.3327
Epoch 16/50
 - 2s - loss: 1.3446 - acc: 0.3403
Epoch 17/50
 - 2s - loss: 1.3475 - acc: 0.3431
Epoch 18/50
 - 2s - loss: 1.3455 - acc: 0.3331
Epoch 19/50
 - 2s - loss: 1.3372 - acc: 0.3459
Epoch 20/50
 - 2s - loss: 1.3362 - acc: 0.3423
Epoch 21/50
 - 2s - loss: 1.3352 - acc: 0.3491
Epoch 22/50
 - 2s - lo